In [1]:
function parameters(lamda::Complex128, mu::Complex128, rho::Float64, omega::Float64, k::Float64)

    alpha = sqrt((lamda+2*mu)/rho)
    beta  = sqrt(mu/rho)

    # P-wave velocity:
    p = sqrt(1-(omega/(k*alpha))^2 + 0im)

    # S-wave velocity:
    s = sqrt(1-(omega/(k*beta))^2  + 0im)

    kp = k*p
    ks = k*s

    return p, s, kp, ks, alpha, beta
end

parameters (generic function with 1 method)

In [2]:
@inline function klayer_h0(k::Float64, omega::Float64, lamda::Complex128, mu::Complex128, rho::Float64)

    (p, s, kp, ks, alpha, beta) = parameters(lamda, mu,  rho,  omega, k)

    # ordered by lines [[1,2][3,4]] -> [1,2,3,4]
    K = Vector{Complex128}(4)

    K[1] = (1-s*s)/(2*(1-p*s))*p
    K[2] = -(1-s*s)/(2*(1-p*s)) + 1
    K[3] = K[2]
    K[4] = (1-s*s)/(2*(1-p*s))*s

    K .*= 2*k*mu

    return K
end

klayer_h0 (generic function with 1 method)

In [3]:
@inline function layer_k(k::Float64, omega::Float64, lamda::Complex128, mu::Complex128, rho::Float64, h::Float64)

    (p, s, kp, ks, alpha, beta) = parameters(lamda, mu,  rho,  omega, k)

    # ordered by lines [[1,2][3,4]] -> [1,2,3,4]
    K = Vector{Complex128}(16)

    a = real(kp) * h
    b = imag(kp) * h
    c = real(ks) * h
    d = imag(ks) * h

    C1 = 0.5*( (1+exp(-2*a))*cos(b) +1im*(1-exp(-2*a))*sin(b) )
    C2 = 0.5*( (1+exp(-2*c))*cos(d) +1im*(1-exp(-2*c))*sin(d) )
    S1 = 0.5*( (1-exp(-2*a))*cos(b) +1im*(1+exp(-2*a))*sin(b) )
    S2 = 0.5*( (1-exp(-2*c))*cos(d) +1im*(1+exp(-2*c))*sin(d) )

    denom = 2*(exp(-a-c)-C1*C2) + (1/(p*s) +p*s)*S1*S2

    # K:

    # K11  K12
    # K21  K22

    # 0   1   2   3  #
    # 4   5   6   7  #
    # 8   9   10  11 #
    # 12  13  14  15 #

    # K11

    K[1] = (1-s*s)*0.5 * ((1/s)*C1*S2 - p*C2*S1)/denom
    K[2] = (1-s*s)*0.5 * ( exp(-a-c) - C1*C2 + p*s*S1*S2)/denom + (1+s*s)/2
    K[5] = K[2]
    K[6] = (1-s*s)*0.5 *((1/p)*C2*S1 - s*C1*S2)/denom

    # K22 same as K11 with off-diagonal signs reversed
    K[11] =  K[1]
    K[12] = -K[2]
    K[15] = -K[5]
    K[16] =  K[6]

    # K12
    K[3] = (1-s*s) * 0.5*(p*S1*exp(-c) - (1/s)*S2*exp(-a))/denom
    K[4] = (1-s*s) * 0.5*( C1*exp(-c) - C2*exp(-a) )/denom
    K[7] = -K[4]
    K[8] = (1-s*s) * 0.5*(s*S2*exp(-a) - (1/p)*S1*exp(-c))/denom

    # K21
    K[9]  = K[3]
    K[10] = K[7]
    K[13] = K[4]
    K[14] = K[8]

    K .*= 2*k*mu

    return K
end

layer_k (generic function with 1 method)

In [4]:
function fc_soil_multilayer_matrix(k::Float64, omega::Float64, lamda::Vector{Complex128}, G::Vector{Complex128}, rho::Vector{Float64}, h::Vector{Float64})

    # number of layers:
    n_layers = length(h)

    # size of the system of equations for the SP-waves problem:
    size_SP_system = 2*n_layers

    # preallocations
    # int ii, jj, kk, iii,jjj
    cols   = Vector{Int64}(16)
    colsh0 = Vector{Int64}(4)
    rows   = Vector{Int64}(16)
    rowsh0 = Vector{Int64}(4)

    col    = Vector{Int64}()
    row    = Vector{Int64}()

    K      = zeros(Complex128, size_SP_system, size_SP_system)
    invK   = zeros(Complex128, size_SP_system, size_SP_system)


    #cdef double complex nc

    klayer0 = Vector{Complex128}(4)
    klaery  = Vector{Complex128}(16)

    #K = ndarray(shape=(size_SP_system,size_SP_system), dtype=float, order='F')

    rows = [0,0,0,0,1,1,1,1,2,2,2,2,3,3,3,3]
    cols = [0,1,2,3,0,1,2,3,0,1,2,3,0,1,2,3]

    rowsh0 = [0,0,1,1]
    colsh0 = [0,1,0,1]

    @inbounds for ii in 1:n_layers

        if h[ii]==0

            klayerh0 = klayer_h0(k, omega,  lamda[ii], G[ii], rho[ii])

            # Assembly in global matrix:
            for jj in 1:4

                nc = klayerh0[jj]
                iii = rowsh0[jj]+2*ii-1
                jjj = colsh0[jj]+2*ii-1
                K[iii, jjj] .+= nc
            end

        else

            klayer = layer_k(k, omega, lamda[ii], G[ii], rho[ii], h[ii])

            # Assembly in global matrix:
            for jj in 1:16

                nc = klayer[jj]
                iii = rows[jj]+2*ii-1
                jjj = cols[jj]+2*ii-1
                K[iii, jjj] .+= nc
            end
        end
    end

    invK = inv(K)

    flex_coef = invK[2, 2]

    return K, flex_coef
end

fc_soil_multilayer_matrix (generic function with 1 method)

In [5]:
bb    = [1,2,3,4,5]
k     = 0.5
omega = 1.
lamda = Complex128[1e6,2e6,3e6]
G     = Complex128[1e6,2e6,3e6]/2.
rho   = Float64[2e3,2e3,2e3]
h     = [1.,1,0]
K, flex_coef = fc_soil_multilayer_matrix(k, omega, lamda, G, rho, h)

(Complex{Float64}[633096.0+0.0im -98376.1+0.0im … 0.0+0.0im 0.0+0.0im; -98376.1+0.0im 1.95501e6+0.0im … 0.0+0.0im 0.0+0.0im; … ; 0.0+0.0im 0.0+0.0im … 2.46526e6+0.0im 4.97401e5+0.0im; 0.0+0.0im 0.0+0.0im … 4.97401e5+0.0im 5.1068e6+0.0im], 1.3562693721523242e-6 + 0.0im)

In [6]:
flex_coef

1.3562693721523242e-6 + 0.0im

In [7]:
using TimeIt

In [8]:
@timeit fc_soil_multilayer_matrix(k, omega, lamda, G, rho, h)

10000 loops, best of 3: 

89.42 µs per loop


8.94207791e-5

In [9]:
using Compat, BenchmarkTools

In [10]:
@benchmark fc_soil_multilayer_matrix(k, omega, lamda, G, rho, h)

BenchmarkTools.Trial: 
  memory estimate:  18.27 KiB
  allocs estimate:  208
  --------------
  minimum time:     13.533 μs (0.00% GC)
  median time:      17.397 μs (0.00% GC)
  mean time:        116.401 μs (1.88% GC)
  maximum time:     91.776 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1